In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

import warnings
warnings.simplefilter('ignore')

In [2]:
#!pip install implicit

In [3]:
data = pd.read_csv('./retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [5]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [6]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [7]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар? 
##### Все остальные товары идущие после 5000 самых популярных в item_id имеют 999999

Зачем он нужен?
##### чтобы пометить эти товары

Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?
##### В сторону этого значения.  Или что подразумевается под стороной?

Можно ли удалить этот товар?
Уберите этот товар и сравните с качеством на семинаре.
##### Уберет большую часть датасета. Или что подразумевается под убрать?  Поменять это значение или удалить строки с этим товаром? О чем речь конкретно?

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [9]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'].tolist())
    
    return recs.tolist()

##### 1) наиболее часто продающиеся товары

In [10]:
it_0 = data_train.groupby('item_id')['sales_value'].apply(lambda x: np.log1p(x).sum()) / data_train.groupby('item_id')['sales_value'].apply(lambda x: np.log1p(x).sum()).sum()
it_0

item_id
25671       2.066911e-06
26081       2.491315e-07
26093       3.445371e-07
26190       3.374796e-07
26355       3.953187e-07
                ...     
17381856    0.000000e+00
17382205    7.950782e-07
17383227    6.165262e-07
17827644    4.535489e-07
17829232    1.407955e-06
Name: sales_value, Length: 86865, dtype: float64

In [11]:
it_0 = it_0.reset_index()
it_0.columns = ['item_id', 'weight']
it_0.head(3)

,item_id,weight
0,25671,2.066911e-06
1,26081,2.491315e-07
2,26093,3.445371e-07


In [12]:
# Что так, что с log, precision одинаковый практически
# it = data_train.groupby('item_id')['sales_value'].sum().reset_index()
# it['weight'] = it.sales_value / it.sales_value.sum()
# it.drop(['sales_value'], axis=1, inplace=True)

In [13]:
# it.weight.sum()

Сделайте предсказания

In [14]:
%%time

result['WRR_popular'] = result['user_id'].apply(lambda x: weighted_random_recommendation(it_0, n=5))
result.head(5)

CPU times: user 9.76 s, sys: 22.9 ms, total: 9.78 s
Wall time: 9.78 s


,user_id,actual,WRR_popular
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[881594, 1138467, 869900, 1026984, 1028950]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[980823, 967476, 1127831, 9527327, 914957]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[878637, 9858911, 6424471, 9526313, 826385]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1000050, 1109805, 923746, 1015306, 945902]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1037863, 1120741, 1137346, 1092122, 12262992]"


##### 2) по количеству купленного товара

In [15]:
it_2 = data_train.groupby('item_id')['quantity'].sum().reset_index()
it_2['weight'] = it_2.quantity / it_2.quantity.sum()
it_2.drop(['quantity'], axis=1, inplace=True)

In [16]:
%%time

result['WRR_quantity'] = result['user_id'].apply(lambda x: weighted_random_recommendation(it_2, n=5))
result.head(5)

CPU times: user 10.9 s, sys: 28.2 ms, total: 11 s
Wall time: 11 s


,user_id,actual,WRR_popular,WRR_quantity
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[881594, 1138467, 869900, 1026984, 1028950]","[6410462, 6534178, 6544236, 1024306, 1133157]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[980823, 967476, 1127831, 9527327, 914957]","[6534178, 6534166, 397896, 6533889, 5747420]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[878637, 9858911, 6424471, 9526313, 826385]","[6533889, 6534178, 6534166, 1404121, 397896]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1000050, 1109805, 923746, 1015306, 945902]","[6534178, 6534166, 6533889, 911878, 6544236]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1037863, 1120741, 1137346, 1092122, 12262992]","[6534178, 6034964, 6534166, 6544236, 6533889]"


##### 3) по скидкам

In [17]:
data_train['all_discount'] = np.abs(round(data_train['coupon_disc'] + data_train['coupon_match_disc'] + data_train['retail_disc']))

In [18]:
it_3 = data_train.groupby('item_id')['all_discount'].sum().reset_index()
it_3['weight'] = it_3.all_discount / it_3.all_discount.sum()
it_3.drop(['all_discount'], axis=1, inplace=True)

In [19]:
%%time

result['WRR_all_discount'] = result['user_id'].apply(lambda x: weighted_random_recommendation(it_3, n=5))
result.head(5)

CPU times: user 9.9 s, sys: 23.4 ms, total: 9.92 s
Wall time: 9.92 s


,user_id,actual,WRR_popular,WRR_quantity,WRR_all_discount
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[881594, 1138467, 869900, 1026984, 1028950]","[6410462, 6534178, 6544236, 1024306, 1133157]","[942475, 13115915, 5978656, 835922, 1007911]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[980823, 967476, 1127831, 9527327, 914957]","[6534178, 6534166, 397896, 6533889, 5747420]","[946995, 1024306, 14025398, 12695874, 994065]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[878637, 9858911, 6424471, 9526313, 826385]","[6533889, 6534178, 6534166, 1404121, 397896]","[1047923, 904777, 13113121, 993466, 909894]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1000050, 1109805, 923746, 1015306, 945902]","[6534178, 6534166, 6533889, 911878, 6544236]","[821465, 1059902, 916990, 1133018, 13213083]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1037863, 1120741, 1137346, 1092122, 12262992]","[6534178, 6034964, 6534166, 6544236, 6533889]","[978221, 10344585, 1016260, 916122, 9420314]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [20]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [21]:
# result['column_name'].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))[0]

### Модели с урока, общий precision@5 в конце задания.

In [22]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [25]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [26]:
# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
data_train

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,all_discount
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,1.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2282320,222,41297772783,635,1120741,1,0.59,304,0.00,1716,91,0.0,0.0,0.0
2282321,462,41297773713,635,999999,1,1.99,304,0.00,2040,91,0.0,0.0,0.0
2282322,462,41297773713,635,995242,1,1.00,304,-0.89,2040,91,0.0,0.0,1.0
2282323,462,41297773713,635,10180324,1,3.00,304,-0.29,2040,91,0.0,0.0,0.0


In [28]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [29]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

##### Item Item Recommender

In [30]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 1.49 s, sys: 39.3 ms, total: 1.53 s
Wall time: 1.53 s


In [31]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 59.3 ms, sys: 2.66 ms, total: 62 ms
Wall time: 61.2 ms


In [32]:
result.head(3)

,user_id,actual,WRR_popular,WRR_quantity,WRR_all_discount,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[881594, 1138467, 869900, 1026984, 1028950]","[6410462, 6534178, 6544236, 1024306, 1133157]","[942475, 13115915, 5978656, 835922, 1007911]","[999999, 1082185, 981760, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[980823, 967476, 1127831, 9527327, 914957]","[6534178, 6534166, 397896, 6533889, 5747420]","[946995, 1024306, 14025398, 12695874, 994065]","[999999, 1082185, 981760, 1098066, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[878637, 9858911, 6424471, 9526313, 826385]","[6533889, 6534178, 6534166, 1404121, 397896]","[1047923, 904777, 13113121, 993466, 909894]","[999999, 1082185, 981760, 1127831, 995242]"


###### Cosine Recommender

In [33]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


CPU times: user 1.45 s, sys: 34.3 ms, total: 1.48 s
Wall time: 1.48 s


In [34]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 73.6 ms, sys: 2.37 ms, total: 75.9 ms
Wall time: 74.6 ms


###### TF IDF Recommender

In [35]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


CPU times: user 1.47 s, sys: 19.1 ms, total: 1.49 s
Wall time: 1.49 s


In [36]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

CPU times: user 73.6 ms, sys: 2.22 ms, total: 75.8 ms
Wall time: 74.6 ms


In [37]:
for col in result.columns[2:]:
    print(f'{col} precision ---> {result.apply(lambda x: precision_at_k(x[col], x["actual"],  5), axis=1).mean()}')

WRR_popular precision ---> 0.021645445641527938
WRR_quantity precision ---> 0.048383937316356894
WRR_all_discount precision ---> 0.01900097943192949
itemitem precision ---> 0.13692458374142857
cosine precision ---> 0.13290891283055686
tfidf precision ---> 0.1389813907933383


По результатам можно сказать, что по WRR результаты слабее, чем у моделей, потому что рандобный выбор товаров - это естественно. У моделей с данными параметрами результаты практически идентичные, оценить какая лучше, в этом случае, не представляется возможным. Попробуем улучшить в следующем задании

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [38]:
# top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [39]:
# # Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
# data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# user_item_matrix = pd.pivot_table(data_train, 
#                                   index='user_id', columns='item_id', 
#                                   values='quantity',
#                                   aggfunc='count', 
#                                   fill_value=0
#                                  )

# user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
# user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# # переведем в формат saprse matrix
# sparse_user_item = csr_matrix(user_item_matrix).tocsr()

# user_item_matrix.head(3)

##### Item Item Recommender

In [40]:
%%time

model = ItemItemRecommender(K=2, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 1.43 s, sys: 13.1 ms, total: 1.45 s
Wall time: 1.44 s


In [41]:
%%time

result['itemitem_2'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 54.9 ms, sys: 2.4 ms, total: 57.3 ms
Wall time: 56 ms


In [42]:
result.head(3)

,user_id,actual,WRR_popular,WRR_quantity,WRR_all_discount,itemitem,cosine,tfidf,itemitem_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[881594, 1138467, 869900, 1026984, 1028950]","[6410462, 6534178, 6544236, 1024306, 1133157]","[942475, 13115915, 5978656, 835922, 1007911]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 995242, 1029743, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[980823, 967476, 1127831, 9527327, 914957]","[6534178, 6534166, 397896, 6533889, 5747420]","[946995, 1024306, 14025398, 12695874, 994065]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[878637, 9858911, 6424471, 9526313, 826385]","[6533889, 6534178, 6534166, 1404121, 397896]","[1047923, 904777, 13113121, 993466, 909894]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 981760, 995242, 1029743]"


###### Cosine Recommender

In [43]:
%%time

model = CosineRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


CPU times: user 1.45 s, sys: 20.8 ms, total: 1.47 s
Wall time: 1.46 s


In [44]:
%%time

result['cosine_2'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 51.9 ms, sys: 2.22 ms, total: 54.2 ms
Wall time: 52.9 ms


###### TF IDF Recommender

In [45]:
%%time

model = TFIDFRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


CPU times: user 1.45 s, sys: 14.1 ms, total: 1.46 s
Wall time: 1.46 s


In [46]:
%%time

result['tfidf_2'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

CPU times: user 66.7 ms, sys: 2.42 ms, total: 69.1 ms
Wall time: 67.9 ms


In [47]:
for col in result.columns[2:]:
    print(f'{col} precision ---> {round(result.apply(lambda x: precision_at_k(x[col], x["actual"],  5), axis=1).mean()*100,2)}%')

WRR_popular precision ---> 2.16%
WRR_quantity precision ---> 4.84%
WRR_all_discount precision ---> 1.9%
itemitem precision ---> 13.69%
cosine precision ---> 13.29%
tfidf precision ---> 13.9%
itemitem_2 precision ---> 19.2%
cosine_2 precision ---> 17.28%
tfidf_2 precision ---> 14.64%


После изменения количества ближайших соседей в моделях ситуация немного изменилась, лучшим оказался ItemItemRecommender с k=2.

### Задание 4 *. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



In [ ]:
# n = 10

In [53]:
def cossim(a, b): numpy.inner(a, b)/(numpy.linalg.norm(a)*numpy.linalg.norm(b)) # cosine similarity

In [ ]:
# for i in range(n):
#     cossim(matrix[i], matrix[i+1])

In [ ]:
# for i in range(n): # например по каждому из соседей
#     for j in ...:
#         sum(cossim(a,b)*b[i][0])/

In [ ]:
# r_u = sum(cossim(a,b)*(b[i][0]))

Данных с вебинара не достаточно для решения этого задания! Также как и не достаточно знаний полученых до, для решения! Сделаем скидку, что наш разговор в чате был после этого урока и этого задания! Надеюсь в 3 уроке уже таких моментов не будет.

Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

В модели вместо рейтингов будут 0 и 1, а рейтинги(значения матрицы) будут штрафом за ошибку в прогнозе (будут выступать в качестве весов ошибки).

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.

In [52]:
?CosineRecommender